# RAG пайплайн

Этот блокнот предназначен для запуска в ядре виртуального окружения Poetry.
При написании кода использовался cursor с gpt-5.2 codex.

Перед запуском заполните, пожалуйста, файл .env.example c GIGACHAT_API_KEY и PINECONE_API_KEY.
Их можно получить бесплатно на https://developers.sber.ru/ и https://app.pinecone.io 

## Настройка

Окружение, пути к файлам, импорты.

In [1]:
import os, glob, pathlib, json, time, re
from typing import List, Dict, Optional

# API's
from pathlib import Path
from dotenv import load_dotenv

ENV_PATH = Path.cwd() / ".env"
load_dotenv(ENV_PATH)

# Sanity checks
for k in ["PINECONE_API_KEY", "GIGACHAT_API_KEY"]:
    assert os.getenv(k), f"Missing {k} env var"

# Paths
PROJECT_ROOT = Path.cwd()
DATA_DIR = str(PROJECT_ROOT / "articles")   # путь к папке с pdf статьями для retrieval
TMP_DIR = str(PROJECT_ROOT / "articles_md")
CITATIONS_XLSX = str(PROJECT_ROOT / "alzheimer_citations.xlsx")
pathlib.Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
pathlib.Path(TMP_DIR).mkdir(parents=True, exist_ok=True)

# Citation map: pdf filename -> citation
import pandas as pd

_citations_df = pd.read_excel(CITATIONS_XLSX)
_citations_df = _citations_df.dropna(subset=["pdf file name", "citation"])
_citation_map = {
    str(row["pdf file name"]).strip().lower(): str(row["citation"]).strip()
    for _, row in _citations_df.iterrows()
}

def get_citation(pdf_filename: str) -> str:
    key = str(pdf_filename).strip().lower()
    citation = _citation_map.get(key)
    if not citation:
        raise RuntimeError(f"Missing citation for PDF: {pdf_filename}")
    return citation


## Конвертация pdf в markdown
Используется конвертер Docling.

In [2]:
try:
    from docling.document_converter import DocumentConverter

    _docling_converter = DocumentConverter()

    def docling_parse(pdf_path: str) -> str:
        t0 = time.time()
        print(f"[docling] converting: {pdf_path}")
        result = _docling_converter.convert(pdf_path)
        print(f"[docling] converted in {time.time() - t0:.2f}s: {pdf_path}")
        md = result.document.export_to_markdown()
        print(f"[docling] markdown exported in {time.time() - t0:.2f}s: {pdf_path}")
        return md

except Exception as e:
    raise RuntimeError(
        "Docling import failed. Ensure `docling` is installed."
    ) from e

def parse_folder_to_md(input_dir: str, out_dir: str) -> List[Dict[str, str]]:
    out = []
    pdfs = glob.glob(str(pathlib.Path(input_dir) / "**/*.pdf"), recursive=True)
    print(f"[parse] found {len(pdfs)} pdf(s) in {input_dir}")
    for idx, pdf in enumerate(pdfs, start=1):
        stem = pathlib.Path(pdf).stem
        md_path = str(pathlib.Path(out_dir) / f"{stem}.md")
        if pathlib.Path(md_path).exists():
            print(f"[parse] ({idx}/{len(pdfs)}) skipping (already converted): {md_path}")
            out.append({"source_pdf": pdf, "markdown_path": md_path})
            continue

        print(f"[parse] ({idx}/{len(pdfs)}) parsing: {pdf}")
        t0 = time.time()
        md = docling_parse(pdf)
        print(f"[parse] ({idx}/{len(pdfs)}) docling_parse took {time.time() - t0:.2f}s")
        with open(md_path, "w", encoding="utf-8") as f:
            f.write(md)
        print(f"[parse] wrote markdown: {md_path}")
        out.append({"source_pdf": pdf, "markdown_path": md_path})
    return out

parsed = parse_folder_to_md(DATA_DIR, TMP_DIR)
print(f"Parsed PDFs: {len(parsed)}")
if parsed[:3]:
    print(parsed[:3])

[parse] found 10 pdf(s) in /home/demid/biocad/test/articles
[parse] (1/10) skipping (already converted): /home/demid/biocad/test/articles_md/gleason2020.md
[parse] (2/10) skipping (already converted): /home/demid/biocad/test/articles_md/garciaosta2012.md
[parse] (3/10) skipping (already converted): /home/demid/biocad/test/articles_md/grill2010.md
[parse] (4/10) skipping (already converted): /home/demid/biocad/test/articles_md/weiming2023.md
[parse] (5/10) skipping (already converted): /home/demid/biocad/test/articles_md/penke2023.md
[parse] (6/10) skipping (already converted): /home/demid/biocad/test/articles_md/valenza2021.md
[parse] (7/10) skipping (already converted): /home/demid/biocad/test/articles_md/teeba2021.md
[parse] (8/10) skipping (already converted): /home/demid/biocad/test/articles_md/sunho2014.md
[parse] (9/10) skipping (already converted): /home/demid/biocad/test/articles_md/gong2018.md
[parse] (10/10) skipping (already converted): /home/demid/biocad/test/articles_md/se

Создание langchain documents.

In [3]:
from langchain_core.documents import Document

# list of documents (wraped in langchain Document)
docs: List[Document] = []

# Read the Markdown file
for item in parsed:
    with open(item["markdown_path"], "r", encoding="utf-8") as f:
        text = f.read()

    pdf_name = pathlib.Path(item["source_pdf"]).name
    citation = get_citation(pdf_name)
    docs.append(
        Document(
            page_content=text,
            metadata={
                "citation": citation,
                "source": item["source_pdf"],
            },
        )
    )

# preview
docs

[Document(metadata={'citation': "Gleason A, Bush AI. Iron and Ferroptosis as Therapeutic Targets in Alzheimer's Disease. Neurotherapeutics. 2021 Jan;18(1):252-264. doi: 10.1007/s13311-020-00954-y. Epub 2020 Oct 27. PMID: 33111259; PMCID: PMC8116360.", 'source': '/home/demid/biocad/test/articles/gleason2020.pdf'}, page_content="## CURRENT PERSPECTIVES\n\n## Iron and Ferroptosis as Therapeutic Targets in Alzheimer ' s Disease\n\nAndrew Gleason 1 &amp; Ashley I. Bush 1\n\n<!-- image -->\n\nAccepted: 17 October 2020 # The American Society for Experimental NeuroTherapeutics, Inc. 2020 / Published online: 27 October 2020\n\n## Abstract\n\nAlzheimer ' s disease (AD), one of the most common neurodegenerative diseases worldwide, has a devastating personal, familial, and societal impact. In spite of profound investment and effort, numerous clinical trials targeting amyloidβ , which is thought to have a causative role in the disease, have not yielded any clinically meaningful success to date. Iro

### Эксплораторный анализ текстов


In [4]:
import statistics

all_lines = []
for d in docs:
    all_lines.extend(d.page_content.splitlines())

header_lines = [ln for ln in all_lines if ln.strip().startswith("#")]
paragraphs = [p.strip() for p in "\n".join(all_lines).split("\n\n") if p.strip()]
para_lengths = [len(p) for p in paragraphs]

print(f"Всего строк: {len(all_lines)}")
print(f"Строк с заголовками: {len(header_lines)}")
print(f"Строк с абзацами: {len(paragraphs)}")
print(f"Медианная длина абзаца (символов): {int(statistics.median(para_lengths)) if para_lengths else 0}")
print(f"95th percentile абзаца (символов): {int(statistics.quantiles(para_lengths, n=20)[-1]) if len(para_lengths) >= 20 else 0}")

# Show a few common header levels
header_samples = [h.strip() for h in header_lines[:5]]
print("Примеры заголовков:")
for h in header_samples:
    print("-", h)

Всего строк: 4475
Строк с заголовками: 199
Строк с абзацами: 1261
Медианная длина абзаца (символов): 257
95th percentile абзаца (символов): 3582
Примеры заголовков:
- ## CURRENT PERSPECTIVES
- ## Iron and Ferroptosis as Therapeutic Targets in Alzheimer ' s Disease
- ## Abstract
- ## Introduction
- ## Iron and Oxidative Stress in the Brain


**Вывод для дальнейшего чанкирования:**
В статьях содержатся частые заголовки разделов и небольшие абзацы, поэтому разбиваем текст по заголовкам `##/###/####` и границам абзацев, чтобы сохранить смысловую связность. Мы держим фрагменты примерно по 400 токенов, чтобы дробить только большие разделы, ну и это стандартная практика. В качестве разделителей также учитываются начала списков и таблиц, чтобы они выделялись в отдельные чанки. Пересечение чанков в 50 токенов - стандартная практика, пересечение слабо влияет на качество retrieval.


## Загрузка данных

Чанкирование. Ниже выведен один из чанков.

In [5]:
import tiktoken
from langchain_text_splitters import RecursiveCharacterTextSplitter

encoding = tiktoken.get_encoding("cl100k_base")

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name=encoding.name,  
    chunk_size=400,
    chunk_overlap=50,
    separators=[
        # "\n```",                    # keep fenced code blocks intact
        # "\n# ",                     # H1
        "\n## ",                    # H2
        "\n### ",                   # H3
        "\n#### ",                  # H4
    
        "\n\n",                     # paragraphs
        "\n- ", "\n* ", "\n1. ",    # lists
        "\n|",                      # table row starts (helps keep rows together)
        "\n",                       # line breaks
        ". ",                       # sentence-ish
        " ",
        ""
    ],
)

chunks: List[Document] = splitter.split_documents(docs)
print("Всего чанков:", len(chunks))
print(chunks[20])

Всего чанков: 901
page_content='## Table 1 Ferroptosis inhibitors

Lipid peroxidation inhibitors

- Vitamin E
- Deuterated polyunsaturated fatty acids
- Butylated hydroxytoluene
- Butylated hydroxyanisole
- Ferrostatins
- Liproxstatins
- Coenzyme Q10
- Idebenone
- Gliptins (vidagliptin, alogliptin, linagliptin)

Iron depleters

- Deferoxamine
- Cyclipirox
- Deferiprone

Lipoxygenase-induced lipid peroxidation inhibitors

- CDC (cinnamyl-3,4-dihydroxyα -cyanocinnamate)
- Baicalein
- PD-146176
- AA-861
- Zileuton

System Xc- inhibitor blockers

- Cycloheximide
- 2-Mercaptoethanol

Blocker of GPX4 degradation

- Dopamine

Increased selenoproteins

- Selenium

Modified from Stockwell et al. [11]

in 3xTg AD mice [136, 137]. A small randomized controlled pilot trial of supra-nutritional sodium selenate in AD showed that subjects whose CSF selenium concentrations raised with treatment had reduced deterioration on the Mini-Mental Status Examination (MMSE) [138].' metadata={'citation': "Gleaso

Первые 3 чанка и их длины в токенах.

In [6]:
from helpers import analyze_chunks_df

chunk_texts = [c.page_content for c in chunks]
analyze_chunks_df(chunk_texts, use_tokens=True).head(3)

,chunk #,text,overlap,# tokens
0,1st,## CURRENT PERSPECTIVES\n\n## Iron and Ferropt...,,311
1,2nd,## Introduction\n\nAround 50 million people ha...,,287
2,3rd,<!-- image -->\n\namyloid cascade hypothesis i...,,322


Визуализация первых 3 чанков.

In [7]:
from IPython.display import HTML
from helpers import colorize_chunks_markdown

chunk_texts = [c.page_content for c in chunks]
HTML(colorize_chunks_markdown(chunk_texts[:3]))

## Индексирование
Будем хранить эмбеддинги чанков на сервере Pinecone.

Используем гибридный поиск, который одновременно использует dense и sparse эмбеддинги. Гибридный поиск считает сходство как взвешенную комбинацию скалярного произведения query с dense эмбеддингом и sparse score чанка и query. Dense эмбеддинги хорошо сохраняют смысл текстов, а sparse score хорош для имен, аббревиатур и точных терминов, которых в научных статьях много. 

Dense делаем с помощью sentence-transformers/all-MiniLM-L6-v2 - это небольшая и быстрая модель, которая даёт приемлимое качество эмбеддингов.

Sparse score считается с помощью BM25 - он быстрый и не требует обучения. Скор считается на основании tf, idf и длины чанка. Считается хорошей моделью.

Создаем pinecone индекс.

In [8]:
from pinecone import Pinecone, ServerlessSpec
from tqdm import tqdm
import time

def _ts(msg: str) -> None:
    print(f"[{time.strftime('%H:%M:%S')}] {msg}")

# Pinecone config
INDEX_NAME = "rag-hybrid-agentic-local-384"
PC_CLOUD = "aws"
PC_REGION = "us-east-1"     # where serverless is available

_ts("Init Pinecone client")
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
existing = {ix["name"]: ix for ix in pc.list_indexes()}
if INDEX_NAME not in existing:
    _ts(f"Creating index: {INDEX_NAME}")
    pc.create_index(
        name=INDEX_NAME,
        dimension=384,          # sentence-transformers/all-MiniLM-L6-v2
        metric="dotproduct",   # supports sparse+dense hybrid
        spec=ServerlessSpec(cloud=PC_CLOUD, region=PC_REGION),
    )
index = pc.Index(INDEX_NAME)

[20:59:37] Init Pinecone client


Создаем и добавляем в индекс эмбеддинги и sparse представления чанков. Но только те, что там отсутствуют, чтобы при каждом новом запуске ноутбука с одними и теми же статьями не делать это заново.

In [9]:

from pinecone_text.sparse import BM25Encoder


# 1. embedding (local sentence-transformers)
from sentence_transformers import SentenceTransformer

LOCAL_EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # 384-dim
_ts(f"Using local embeddings: {LOCAL_EMBED_MODEL}")
encoder = SentenceTransformer(LOCAL_EMBED_MODEL)

# 2. sparse
texts = [d.page_content for d in chunks]
bm25 = BM25Encoder()
_ts("Fitting BM25")
bm25.fit(texts)

# Batch embeddings + sparse encoding to show progress and allow interrupts
batch_size = 32
_ts("Embedding documents")
dense_vecs = []
for i in tqdm(range(0, len(texts), batch_size), desc="Embed batches"):
    t0 = time.time()
    batch = texts[i : i + batch_size]
    try:
        dense_vecs.extend(encoder.encode(batch, normalize_embeddings=True).tolist())
    except Exception as exc:
        raise RuntimeError(f"Embedding batch failed at offset {i}: {exc}")

_ts("Encoding sparse (BM25)")
sparse_vecs = []
for i in tqdm(range(0, len(texts), batch_size), desc="BM25 encode batches"):
    sparse_vecs.extend(bm25.encode_documents(texts[i : i + batch_size]))

import hashlib

# IDs are deterministic, so re-running safely overwrites (idempotent upsert).
def make_id(doc):
    # stable id from content + common metadata fields
    base = f"{doc.metadata.get('source','')}|{doc.metadata.get('md','')}|{doc.page_content}"
    return hashlib.sha256(base.encode("utf-8")).hexdigest()[:32]


to_upsert = []
for doc, dense, sparse in tqdm(
    zip(chunks, dense_vecs, sparse_vecs),
    total=len(chunks),
    desc="Build vectors",
):
    doc_id = doc.metadata.get("id") or make_id(doc)
    doc.metadata["id"] = doc_id  # persist chunk_id for reuse

    citation = doc.metadata.get("citation")
    if not citation or str(citation).strip().lower() == "unknown":
        src_pdf = doc.metadata.get("source")
        if src_pdf:
            citation = get_citation(pathlib.Path(src_pdf).name)
    if not citation or str(citation).strip().lower() == "unknown":
        raise RuntimeError(f"Missing citation for source: {doc.metadata.get('source')}")

    metadata = {
        **doc.metadata,
        "citation": str(citation).strip(),
        "context": doc.page_content,  # IMPORTANT: retriever expects "context"
    }
    to_upsert.append({
        "id": doc_id,
        "values": dense,
        "sparse_values": sparse,      # {"indices":[...], "values":[...]}
        "metadata": metadata,
    })

# Upsert to Pinecone (only missing IDs)
current_pdfs = sorted({pathlib.Path(d.metadata.get("source", "")).name.lower() for d in docs if d.metadata.get("source")})
ns_hash = hashlib.sha1("|".join(current_pdfs).encode("utf-8")).hexdigest()[:8]
namespace = f"docs_{ns_hash}"
print(f"[pinecone] namespace: {namespace}")

# 1) Check which IDs already exist in Pinecone
_ts("Checking existing IDs")
ids = [v["id"] for v in to_upsert]
existing_ids = set()
needs_refresh = set()  # exists but missing/unknown citation
check_batch = 100
for i in tqdm(range(0, len(ids), check_batch), desc="Check existing IDs"):
    batch_ids = ids[i : i + check_batch]
    fetched = index.fetch(ids=batch_ids, namespace=namespace)
    for _id, vec in fetched.get("vectors", {}).items():
        existing_ids.add(_id)
        cit = (vec.get("metadata") or {}).get("citation")
        if not cit or str(cit).strip().lower() == "unknown":
            needs_refresh.add(_id)

# 2) Keep only missing or needing refresh
ids_to_upsert = set(ids) - (existing_ids - needs_refresh)
missing = [v for v in to_upsert if v["id"] in ids_to_upsert]
print(f"[pinecone] {len(existing_ids)} existing, {len(missing)} missing/refresh in '{namespace}'")

# 3) Upsert missing vectors in batches
_ts("Upserting missing vectors")
batch = 100
for i in tqdm(range(0, len(missing), batch), desc="Upsert batches"):
    index.upsert(vectors=missing[i:i+batch], namespace=namespace)


[20:59:38] Using local embeddings: sentence-transformers/all-MiniLM-L6-v2
[20:59:41] Fitting BM25


  0%|          | 0/901 [00:00<?, ?it/s]

[20:59:45] Embedding documents


Embed batches: 100%|██████████| 29/29 [01:02<00:00,  2.16s/it]


[21:00:48] Encoding sparse (BM25)


Build vectors: 100%|██████████| 901/901 [00:00<00:00, 63581.07it/s]


[pinecone] namespace: docs_3f806510
[21:00:51] Checking existing IDs


Check existing IDs: 100%|██████████| 10/10 [01:20<00:00,  8.02s/it]


[pinecone] 891 existing, 0 missing/refresh in 'docs_3f806510'
[21:02:11] Upserting missing vectors


Upsert batches: 0it [00:00, ?it/s]


Создаём сам гибридный retriever с коэффициентом взвешивания 0.5 - то есть скор равен среднему между dense score и sparse score. Retriever будет доставать 5 наиболее ралевантных чанков для query.

In [10]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from langchain_community.embeddings import HuggingFaceEmbeddings

# Use the same local model used for indexing (384-dim)
query_embeddings = HuggingFaceEmbeddings(model_name=LOCAL_EMBED_MODEL)

retriever = PineconeHybridSearchRetriever(
    embeddings=query_embeddings,
    sparse_encoder=bm25,
    index=index,
    namespace=namespace,
    alpha=0.5,     # 0=sparse only, 1=dense only
    top_k=5,       # number of chunks retrieved
)

/tmp/ipykernel_6118/3713064325.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  query_embeddings = HuggingFaceEmbeddings(model_name=LOCAL_EMBED_MODEL)


## LLM Query functions

Используем gigachat lite api, чтобы не скачивать большие веса моделей.
На каждый вопрос будет даваться ответ с указанием источников. В системном промпте прошу LLM использовать только retrieved чанки и ничего не придумывать.

In [11]:
import os
from typing import Dict, Any
from gigachat import GigaChat

from dotenv import load_dotenv
load_dotenv(ENV_PATH)

GIGACHAT_MODEL = "GigaChat"
GIGACHAT_API_KEY = os.getenv("GIGACHAT_API_KEY")
if not GIGACHAT_API_KEY:
    raise RuntimeError("Missing GIGACHAT_API_KEY in environment (.env)")

gigachat_rag = GigaChat(credentials=GIGACHAT_API_KEY, verify_ssl_certs=False, timeout=60)
gigachat_norag = GigaChat(credentials=GIGACHAT_API_KEY, verify_ssl_certs=False, timeout=60)

SYSTEM_PROMPT = (
    "You are a friendly chatbot assistant for biomedical scientists. "
    "Provided context may help answering the question. "
    "Cite sources using the numeric labels shown in the context, like [1], [2]. "
    "Reuse the same number if multiple chunks are from the same source. "
    "Each factual sentence must include at least one citation. "
    "End with a 'Sources:' list where each line starts with the label, e.g., [1] <citation> (no extra numbering)."
)

SYSTEM_PROMPT_NORAG = (
    "You are a friendly chatbot assistant for biomedical scientists."
)


def _ask_gigachat(question: str, context: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Question: {question}\n\nContext:\n{context}"},
    ]
    resp = gigachat_rag.chat({"model": GIGACHAT_MODEL, "messages": messages, "temperature": 0})
    return resp.choices[0].message.content


def _ask_gigachat_norag(question: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT_NORAG},
        {"role": "user", "content": question},
    ]
    resp = gigachat_norag.chat({"model": GIGACHAT_MODEL, "messages": messages, "temperature": 0})
    return resp.choices[0].message.content


def require_citation(meta: Dict[str, Any]) -> str:
    src = (meta or {}).get("citation")
    if not src or str(src).strip().lower() == "unknown":
        raise RuntimeError(f"Missing citation in metadata for source: {(meta or {}).get('source')}")
    return str(src).strip()


def format_context(docs: List[Document]) -> str:
    ctx = []
    source_to_label = {}
    next_id = 1
    for d in docs:
        src = require_citation(d.metadata)
        if src not in source_to_label:
            source_to_label[src] = str(next_id)
            next_id += 1
        label = source_to_label[src]
        ctx.append(f"[{label}] {src}\n{d.page_content[:1000]}")
    s = "\n\n".join(ctx)
    return s

def query(question: str) -> Dict[str, Any]:
    # Step 1: Hybrid retrieval
    retrieved = retriever.invoke(question)
    # Step 2: Compose context and generate
    ctx = format_context(retrieved)
    answer = _ask_gigachat(question, ctx)

    return {
        "answer": answer,
        "retrieved": retrieved,
    }

Посмотрим, как выглядит ответ LLM, и убедимся, что источники цитируются.

In [12]:
# Quick check: RAG answer with citations on first specific question
TEST_DATA_JSON = str(PROJECT_ROOT / "questions" / "test_data_specific.json")

with open(TEST_DATA_JSON, "r", encoding="utf-8") as f:
    data = json.load(f)

# Current format: list of {pdf_name, pairs:[{question, excerpt}, ...]}
first_q = None
if isinstance(data, list) and data and isinstance(data[0], dict):
    pairs = data[0].get("pairs") or []
    if pairs:
        first_q = pairs[0].get("question")

if not first_q:
    raise RuntimeError("No question found in test_data_specific.json")

res = query(first_q)
print("Question:", first_q)
print("\nAnswer:\n", res["answer"])

Question: Which PDE inhibitor was first shown to restore both memory deficits and hippocampal LTP in transgenic mouse models?

Answer:
 The PDE inhibitor **rolitab** was first shown to restore both memory deficits and hippocampal long-term potentiation (LTP) in transgenic mouse models.

[1]

**Rolipram**, a PDE4 inhibitor, demonstrated significant effects in improving cognitive function and enhancing synaptic plasticity in animal models of Alzheimer’s disease [1].

Sources:
[1] García-Osta A, Cuadrado-Tejedor M, García-Barroso C, Oyarzábal J, Franco R. Phosphodiesterases as therapeutic targets for Alzheimer's disease. ACS Chem Neurosci. 2012 Nov 21;3(11):832-44. doi: 10.1021/cn3000907. Epub 2012 Oct 1. PMID: 23173065; PMCID: PMC3503343.


## Метрики

Качество retrieval и ответов.

Качество retrieval оценивается двумя способами.

1. В каждой статье выбирается отрывок, на него задается вопрос, и смотрим, перекрывается ли данный отрывок на 60% хотя бы в одном из retrieved чанков. И считаем среднюю долю по всем вопросам. Для каждой статьи бралось два таких вопроса. Здесь перекрытие - это доля токенов отрывка длиной не менее 4 символов в чанке.  Игнорируем токены длиной 1–3, потому что они слишком частые и несут мало смысла (союзы, предлоги, обрывки). То есть перекрытие - это мера смысловой близости.

2. Во втором способе оценивается работа пайплайна в том числе и на общих вопросах, для которых важны разные участки разных статей. Например, "What are potential targets for Alzheimer's disease treatment?". Для вопроса и каждого чанка спрашиваем вспомогательную LLM, "релевантен ли данный чанк для данного вопроса?" Это субъективная метрика, и лучше было бы использовать для этого человека-эксперта, но такой возможности нет.


Так же оценивается сам ответ на groundedness - насколько ответ использует каждый чанк. Для этого тоже делается отдельный запрос к вспомогательной LLM. Важно заметить, что у этой метрики помими субъективности есть другой недостаток: если чанки не релевантны, то высокое значение этой метрики говорит о нелеревантном ответе.

Далее функции, считающие метрики, и вспомогательные классы и функции.

In [13]:
from dataclasses import dataclass
from typing import Optional, List


@dataclass
class RetrievedChunk:
    text: str
    citation: str


@dataclass
class Info:
    question: str
    chunks: List[RetrievedChunk]
    extract: Optional[str] = None
    citations: Optional[List[str]] = None
    answer: Optional[str] = None


def _normalize_text(s: str) -> str:
    s = s.lower()
    # remove PDF ligature artifacts like /uniFB01
    s = re.sub(r"/unifb\w+", "", s)
    # keep letters/numbers and spaces
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    return " ".join(s.split())


def _token_coverage(extract: str, chunk: str) -> float:
    extract_tokens = [t for t in _normalize_text(extract).split() if len(t) > 3]
    if not extract_tokens:
        return 0.0
    chunk_tokens = set(_normalize_text(chunk).split())
    hit = sum(1 for t in extract_tokens if t in chunk_tokens)
    return hit / len(extract_tokens)


def _citation_match(info_citations: Optional[List[str]], chunk: RetrievedChunk) -> bool:
    if not info_citations:
        return True
    chunk_cit = chunk.citation.strip().lower()
    return any(cit.strip().lower() == chunk_cit for cit in info_citations)


def build_infos_from_json(json_path: str, top_k: int = 5) -> List[Info]:
    with open(json_path, "r", encoding="utf-8") as f:
        items = json.load(f)

    infos: List[Info] = []
    for item in items:
        # Specific format: {pdf_name, pairs: [{excerpt, question}, ...]}
        if isinstance(item, dict) and "pairs" in item:
            pdf_name = item.get("pdf_name", "")
            citation = get_citation(pdf_name) if pdf_name else None
            for pair in item.get("pairs", []):
                question = pair["question"]
                extract = pair.get("excerpt")
                infos.append(_build_info(question, extract, [citation] if citation else None, top_k))
            continue

        # General format: {question, helpful_papers}
        if isinstance(item, dict) and "question" in item:
            question = item["question"]
            helpful = item.get("helpful_papers") or []
            citations = [get_citation(p) for p in helpful]
            infos.append(_build_info(question, None, citations or None, top_k))
            continue

        # Legacy: list[str]
        if isinstance(item, str):
            infos.append(_build_info(item, None, None, top_k))

    return infos


def _build_info(question: str, extract: Optional[str], citations: Optional[List[str]], top_k: int) -> Info:
    retrieved = retriever.invoke(question)[:top_k]
    chunks = [
        RetrievedChunk(
            text=d.page_content,
            citation=require_citation(d.metadata),
        )
        for d in retrieved
    ]
    return Info(question=question, extract=extract, citations=citations, chunks=chunks)


def recall_at_k(infos: List[Info], min_coverage: float = 0.6) -> float:
    hits = 0
    total = 0
    for info in infos:
        if not info.extract:
            continue
        total += 1
        candidates = [ch for ch in info.chunks if _citation_match(info.citations, ch)]
        if not candidates:
            candidates = info.chunks
        max_cov = max((_token_coverage(info.extract, ch.text) for ch in candidates), default=0.0)
        if max_cov >= min_coverage:
            hits += 1
    return hits / total if total else 0.0


def _judge_relevance(question: str, chunk: RetrievedChunk) -> bool:
    prompt = (
        "You are a strict evaluator. Answer only 'true' or 'false'.\n"
        "Question: {question}\n\n"
        "Chunk: {chunk}\n\n"
        "Is the chunk relevant for answering the question?"
    )
    text = _ask_gigachat(
        question=prompt.format(question=question, chunk=chunk.text[:2000]),
        context="",
    )
    return text.strip().lower().startswith("true")


def avg_relevant_fraction(infos: List[Info]) -> float:
    if not infos:
        return 0.0
    fractions = []
    for info in infos:
        if not info.chunks:
            fractions.append(0.0)
            continue
        relevant = 0
        for ch in info.chunks:
            if _judge_relevance(info.question, ch):
                relevant += 1
        fractions.append(relevant / len(info.chunks))
    return sum(fractions) / len(fractions)


def _judge_used_in_answer(question: str, answer: str, chunk: RetrievedChunk) -> bool:
    prompt = (
        "You are a strict evaluator. Answer only 'true' or 'false'.\n"
        "Question: {question}\n\n"
        "Answer: {answer}\n\n"
        "Chunk: {chunk}\n\n"
        "Does the answer use information from the chunk?"
    )
    text = _ask_gigachat(
        question=prompt.format(question=question, answer=answer, chunk=chunk.text[:2000]),
        context="",
    )
    return text.strip().lower().startswith("true")


def used_fraction(info: Info) -> Optional[float]:
    if not info.answer or not info.chunks:
        return None
    used = 0
    for ch in info.chunks:
        if _judge_used_in_answer(info.question, info.answer, ch):
            used += 1
    return used / len(info.chunks)


def avg_used_fraction(infos: List[Info]) -> float:
    values = [v for v in (used_fraction(i) for i in infos) if v is not None]
    return sum(values) / len(values) if values else 0.0


Подсчёт метрик для вопросов на конкретные отрывки.

Ниже в выводе ячейки есть Info - это один семпл для подсчета метрик - кортеж из отрывка, вопроса, чанков и ответа.

Используется 2 вопроса на каждую статью, то есть всего 20 вопросов. В выводе подробная информация для первых 4 вопросов и кумулятивные метрики по всем вопросам.

In [14]:
TEST_DATA_JSON = str(PROJECT_ROOT / "questions" / "test_data_specific.json")
TOP_K = 5

infos = build_infos_from_json(TEST_DATA_JSON, top_k=TOP_K)

print(f"Loaded {len(infos)} questions from {TEST_DATA_JSON}")

for idx, info in enumerate(infos, 1):
    # Generate answer using RAG and store it on Info
    res = query(info.question)
    info.answer = str(res["answer"])
    info.chunks = [
        RetrievedChunk(
            text=d.page_content,
            citation=require_citation(d.metadata),
        )
        for d in res["retrieved"]
    ]

    extract_hit = None
    max_cov = None
    if info.extract:
        candidates = [ch for ch in info.chunks if _citation_match(info.citations, ch)]
        if not candidates:
            candidates = info.chunks
        covs = [(_token_coverage(info.extract, ch.text), ch) for ch in candidates]
        max_cov = max((c for c, _ in covs), default=0.0)
        extract_hit = max_cov >= 0.6

    used_frac = used_fraction(info)

    if idx <= 4:
        print(f"\n=== Info {idx} ===")
        print("Вопрос:", info.question)
        if info.citations:
            print("Источник:", "; ".join(info.citations))
        if info.extract:
            print("Максимальное перекрытие:", f"{max_cov:.2f}" if max_cov is not None else "-")
            print("Recall@k hit (есть ли чанк с долей перекрытия более 0.6):", extract_hit)
        else:
            print("Есть отрывок:", False)

        print(f"Доля чанков, использованных в ответе (groundedness): {used_frac:.2f}" if used_frac is not None else "Доля чанков, использованных в ответе (groundedness): n/a")

        print("Retrieved чанки:")
        for i, ch in enumerate(info.chunks, 1):
            preview = ch.text.replace("\n", " ")[:120]
            cov = _token_coverage(info.extract, ch.text) if info.extract else None
            cov_str = f" | перекрытие чанка и целевого отрывка={cov:.2f}" if cov is not None else ""
            print(f"  {i}. источник={ch.citation}{cov_str} | {preview}")

avg_recall = recall_at_k(infos, min_coverage=0.6)
avg_rel_frac = avg_relevant_fraction(infos)
avg_used = avg_used_fraction(infos)
print(f"\nДоля вопросов, для которых retriever нашел чанк с перекрытием >=0.6: {avg_recall:.2f}")
print(f"Средняя доля релевантных чанков: {avg_rel_frac:.2f}")
print(f"Средняя доля использованных в ответе чанков (groundedness): {avg_used:.2f}")

Loaded 20 questions from /home/demid/biocad/test/questions/test_data_specific.json

=== Info 1 ===
Вопрос: Which PDE inhibitor was first shown to restore both memory deficits and hippocampal LTP in transgenic mouse models?
Источник: García-Osta A, Cuadrado-Tejedor M, García-Barroso C, Oyarzábal J, Franco R. Phosphodiesterases as therapeutic targets for Alzheimer's disease. ACS Chem Neurosci. 2012 Nov 21;3(11):832-44. doi: 10.1021/cn3000907. Epub 2012 Oct 1. PMID: 23173065; PMCID: PMC3503343.
Максимальное перекрытие: 0.62
Recall@k hit (есть ли чанк с долей перекрытия более 0.6): True
Доля чанков, использованных в ответе (groundedness): 0.80
Retrieved чанки:
  1. источник=García-Osta A, Cuadrado-Tejedor M, García-Barroso C, Oyarzábal J, Franco R. Phosphodiesterases as therapeutic targets for Alzheimer's disease. ACS Chem Neurosci. 2012 Nov 21;3(11):832-44. doi: 10.1021/cn3000907. Epub 2012 Oct 1. PMID: 23173065; PMCID: PMC3503343. | перекрытие чанка и целевого отрывка=0.23 | ## 2. COGNIT

Как видно, retriever достает достаточно информативные чанки.

Подсчёт метрик для общих вопросов.

In [15]:
GENERAL_JSON = str(PROJECT_ROOT / "questions" / "test_data_general.json")

infos_general = build_infos_from_json(GENERAL_JSON, top_k=TOP_K)
print(f"Loaded {len(infos_general)} questions from {GENERAL_JSON}")

for idx, info in enumerate(infos_general, 1):
    # Generate answer using RAG and store it on Info
    res = query(info.question)
    info.answer = str(res["answer"])
    info.chunks = [
        RetrievedChunk(
            text=d.page_content,
            citation=require_citation(d.metadata),
        )
        for d in res["retrieved"]
    ]

    relevant = 0
    for ch in info.chunks:
        if _judge_relevance(info.question, ch):
            relevant += 1
    frac = relevant / len(info.chunks) if info.chunks else 0.0
    used_frac = used_fraction(info)

    if idx <= 4:
        print(f"\n=== General Info {idx} ===")
        print("Вопрос:", info.question)
        print(f"Доля релевантных чанков: {frac:.2f}")
        print(f"Доля использованных в ответе чанков: {used_frac:.2f}" if used_frac is not None else "Доля использованных в ответе чанков: n/a")

avg_rel_frac_general = avg_relevant_fraction(infos_general)
avg_used_general = avg_used_fraction(infos_general)
print(f"\nСредняя доля релевантных чанков (общие вопросы): {avg_rel_frac_general:.2f}")
print(f"Средняя доля использованных в ответе чанков (общие вопросы): {avg_used_general:.2f}")

Loaded 5 questions from /home/demid/biocad/test/questions/test_data_general.json

=== General Info 1 ===
Вопрос: Which symptomatic Alzheimer’s drugs are commonly used, and what targets do they act on?
Доля релевантных чанков: 1.00
Доля использованных в ответе чанков: 0.80

=== General Info 2 ===
Вопрос: What immune and microglia-related pathways are being targeted to slow Alzheimer’s progression?
Доля релевантных чанков: 1.00
Доля использованных в ответе чанков: 1.00

=== General Info 3 ===
Вопрос: How early can Alzheimer’s-related pathology begin before symptoms?
Доля релевантных чанков: 1.00
Доля использованных в ответе чанков: 0.00

=== General Info 4 ===
Вопрос: What non-amyloid mechanisms can potentially contribute to neurodegeneration and cognitive decline?
Доля релевантных чанков: 0.00
Доля использованных в ответе чанков: 0.40

Средняя доля релевантных чанков (общие вопросы): 0.64
Средняя доля использованных в ответе чанков (общие вопросы): 0.64


Для 5 общих вопросов (показаны только первые 4) метрики удовлетворительные.

## Сравнение ответов LLM с retrieved чанками и без.

Ниже видно, насколько сильно помогает информация из статей.

In [16]:
# --------------------------------------------
# Questions for RAG vs no-RAG comparison
# --------------------------------------------
GENERAL_JSON = str(PROJECT_ROOT / "questions" / "test_data_general.json")
with open(GENERAL_JSON, "r", encoding="utf-8") as f:
    _general_questions = json.load(f)

QUESTIONS = [item["question"] for item in _general_questions[:3]]


In [17]:
# --------------------------------------------
# Human comparison output (RAG vs no-RAG)
# --------------------------------------------
for i, q in enumerate(QUESTIONS, 1):
    rag_res = query(q)
    rag_answer = str(rag_res["answer"])
    norag_answer = _ask_gigachat_norag(q)

    print(f"\n=== Q{i} ===")
    print("Question:", q)
    print("\n--- RAG ---")
    print(rag_answer)
    print("\n--- no-RAG ---")
    print(norag_answer)
    print("\n" + "-" * 60)



=== Q1 ===
Question: Which symptomatic Alzheimer’s drugs are commonly used, and what targets do they act on?

--- RAG ---
### Commonly Used Symptomatic Drugs for Alzheimer's Disease and Their Targets

The current standard treatments for Alzheimer's disease primarily involve **cholinesterase inhibitors** and **NMDA-receptor antagonists**, both providing only temporary symptomatic relief [1].

#### Cholinesterase Inhibitors
These drugs work by inhibiting the breakdown of acetylcholine, thereby increasing its levels in the brain. Examples include:
- Donepezil
- Rivastigmine
- Galantamine

#### NMDA-Receptor Antagonists
These drugs aim to modulate glutamate receptors, reducing excitotoxicity associated with neuronal damage. An example is Memantine.

Both classes of drugs target specific pathways involved in memory and cognition [2].

### Sources:
[1] Athar T, Al Balushi K, Khan SA. Recent advances on drug development and emerging therapeutic agents for Alzheimer's disease. Mol Biol Rep. 2

Ответы с RAG более наполнены детальной информацией. Это помогает при конкретных вопросах, когда требуются точные знания, но может ухудшать ответ LLM при вопросах, которые скорее требуют общих знаний.
При разных запросах цитируются разные источники, то есть нет перекоса в сторону одного источника.